In [ ]:
%pip uninstall -y transformers
%pip install git+https://github.com/huggingface/transformers
#%pip install torch
%pip install torch==2.1.2

Found existing installation: transformers 4.39.0.dev0
Uninstalling transformers-4.39.0.dev0:
  Successfully uninstalled transformers-4.39.0.dev0
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-0ejm8nj_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-0ejm8nj_
  Resolved https://github.com/huggingface/transformers to commit c8d98405a8f7b0e5d07391b671dcc61bb9d7bad5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8552478 sha256=dbe303585aaddfa7b8db74d3c232f102a35f38f605f9a9366a10360055f7fc90
  Stored in directory: /tmp/pip-ephem-wheel-cache-v5gw_czo/wheels/c0/14/d6/6c9a5582d2ac1

In [ ]:
!python --version
# %pip list | grep torch
# !module spider torch
!nvidia-smi

Python 3.10.9
Sun Feb 25 23:54:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   28C    P0              24W / 250W |      0MiB / 16384MiB |      0%   E. Process |
|                                         |                      |                  N/A |
+-----------------------------------------+-----------

In [3]:
# Provide path to this notebook
root_path = '/content/drive/MyDrive/UMich Milestone II Project/Final_Code_Submission/Supervised_Notebooks/NER/'

# ***MSFT Phi2***

1. https://huggingface.co/microsoft/phi-2
2. Dataset: https://www.kaggle.com/datasets/joebeachcapital/restaurant-reviews?resource=download

### Helper method

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [ ]:
torch.set_default_device("cuda")


In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# review = """This place is a joke. I asked for a manager because the kid(boy)that answered the phone didn't know how to take down my request to make reservations and could not tell me why the reservations function wasn't functioning in yelp, and told me he did not know how reservations worked on yelp. So this kid put me on hold as he was grabbing a manager and of course he lied because no manager answered. Then when i called back multiple times they disconnected the phone. I wonder how this establishment stays in business hiring low class service people handling customer service requests"""
# prompt = f"""Instruct: Perform NER to return only dish and drink terms in following format "{{'food': [], 'drink': []}}" from following text: {review}\nOutput:"""
# review = "The ambience was good, food was quite good . had Saturday lunch , which was cost effective .\nGood place for a sate brunch. One can also chill with friends and or parents.\nWaiter Soumen Das was really courteous and helpful."
review = "My absolute favorite cafe in the city. Their black and white latte is probably the best I've ever had (not too sweet and just the right amount of foam), soups are always really good (even for non-soup people) and there's just a lot of space to do work. The noise level is perfect, the music is at a perfect level, and I always enjoy when patrons bring their dogs. I'll keep giving them my business for as long as I can."
prompt = f"""Instruct: Perform Named Entity Recognition to return only dish and drink items in following format "{{'food': [], 'drink': []}}" from: {review}\nOutput:"""

inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct: Perform Named Entity Recognition to return only dish and drink items in following format "{'food': [], 'drink': []}" from: My absolute favorite cafe in the city. Their black and white latte is probably the best I've ever had (not too sweet and just the right amount of foam), soups are always really good (even for non-soup people) and there's just a lot of space to do work. The noise level is perfect, the music is at a perfect level, and I always enjoy when patrons bring their dogs. I'll keep giving them my business for as long as I can.
Output: {'food': ['soups'], 'drink': ['black and white latte']}
<|endoftext|>


In [ ]:
review = """The food was amazing (yum pizza), really loved the ham burgers and eggs benedict. Had a good time soaking in the summer with a beer, cheers!"""
prompt = f"""Instruct: Perform NER to return only dish and drink items in following format "{{'food': [], 'drink': []}}" from: {review}\nOutput:"""
inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct: Perform NER to return only dish and drink items in following format "{'food': [], 'drink': []}" from: The food was amazing (yum pizza), really loved the ham burgers and eggs benedict. Had a good time soaking in the summer with a beer, cheers!
Output: {'food': ['pizza', 'ham burgers', 'eggs benedict'], 'drink': ['beer']}
<|endoftext|>


In [ ]:
#review = """We used to eat here about every 2 or 3 months, but have not eaten there on over a decade. We sat on the patio and enjoyed"""
review = """'1.Half of ice cream spilled inside the package only..\n2.I ordered Caremel sutra...But I got Brownie Ice cream\n3.and I cream is in liquid. state like a milkshake....I did not expect this from cream stone at all...I have pic of them with me."""
prompt = f"""Instruct: Return only dish and drink items in following format "{{'food': [], 'drink': []}}" from: {review}\nOutput:"""
inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct: Return only dish and drink items in following format "{'food': [], 'drink': []}" from: '1.Half of ice cream spilled inside the package only..
2.I ordered Caremel sutra...But I got Brownie Ice cream
3.and I cream is in liquid. state like a milkshake....I did not expect this from cream stone at all...I have pic of them with me.
Output: {'food': ['half of ice cream spilled inside the package'], 'drink': []}
<|endoftext|>


In [ ]:
# Function to apply NER to each review
def apply_ner(review):
    #prompt = f"""Apply Named Entity Recognition (NER) to identify and return only food and drink items in the given text. Format the output as "{{'food': [], 'drink': []}}". The text to analyze is: {review}\nOutput:"""
    prompt = f"""Instruct: Perform Named Entity Recognition to return only dish and drink items in following format "{{'food': [], 'drink': []}}" from: {review}\nOutput:"""

    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)
    outputs = model.generate(**inputs, max_length=2000)
    text = tokenizer.batch_decode(outputs)[0]

    try:
        output_index = text.find("Output:")
        # Extract the substring starting from "Output:"
        output_string = text[output_index:]
        output_string = output_string.replace("Output:", "")
        output_string = output_string.replace("<|endoftext|>", "")
        # Convert the output string to a Python dictionary
        output_dict = eval(output_string)
    except:
        print(f"An exception occurred for text: {text}")
        output_dict = None

    return output_dict


### Generate annotations for Kaggle dataset

https://www.kaggle.com/datasets/joebeachcapital/restaurant-reviews?resource=download

In [ ]:
# Specify the file path of your CSV file
csv_file = root_path + '../../Dataset/NER/Kaggle/Restaurant reviews.csv'

# Read the CSV file into a DataFrame
original_df = pd.read_csv(csv_file)

# Define the size of each subsample
subsample_size = 1000

# Determine the number of subsamples needed
num_subsamples = len(original_df) // subsample_size

# Iterate over each subsample
for i in range(num_subsamples):
    # for i in range(8,10):
    # Define the start and end index for the current subsample
    start_index = i * subsample_size
    end_index = (i + 1) * subsample_size

    # Extract the current subsample
    subsample_df = original_df.iloc[start_index:end_index]

    print("Processing subsample ", i+1)
    # perform NER
    subsample_df['ner_results'] = subsample_df['Review'].apply(apply_ner)

    # Save the subsample to a CSV file
    save_path = root_path + f'../../Dataset/NER/Kaggle/subsample_{i+1}.csv'
    subsample_df.to_csv(save_path, index=False)
    print("Subsample saved to subsample_{}.csv".format(i+1))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processing subsample  9


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

An exception occurred for text: Instruct: Perform Named Entity Recognition to return only dish and drink items in following format "{'food': [], 'drink': []}" from: The food was too awesome and quick. Chicken was finger licking good.... Served as hot and too quickly and still had the best taste.😋
Paneer tikka could improve.
Output: {'food': ['food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 'food', 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

An exception occurred for text: Instruct: Perform Named Entity Recognition to return only dish and drink items in following format "{'food': [], 'drink': []}" from: Tasty food. However wrong chutney was sent
Output: import nltk
from nltk.tokenize import word_tokenize

def named_entity_recognition(text):
    tokens = word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    named_entities = nltk.ne_chunk(tagged)
    food_drink = []
    for subtree in named_entities.subtrees():
        if subtree.label() == 'ORGANIZATION' or subtree.label() == 'PERSON':
            food_drink.append(subtree.leaves())
    return {'food': food_drink[0], 'drink': food_drink[1]}

text = "Tasty food. However wrong chutney was sent."
print(named_entity_recognition(text))
<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Subsample saved to subsample_10.csv


/tmp/ipykernel_3663562/2620784753.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample_df['ner_results'] = subsample_df['Review'].apply(apply_ner)


### Generate annotations for Yelp Dataset



In [ ]:
# Yelp dataset (JSON file)
csv_file = root_path + '../../Dataset/philadelphia_restaurant_reviews.json'

# Read the Yelp dataset (JSON file) into a DataFrame
original_df = pd.read_json(csv_file)

In [ ]:
# size of dataframe is > 600k
# Will annotate only first 10k reviews
# original_df = original_df.iloc[:10000]

# Will annotate only next 10k reviews
original_df = original_df.iloc[10000:20000]
FILE_OFFSET = 10

In [ ]:
original_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
10000,ik_4nbr_iv7SEL4H5WS05w,zLl8E4adoZ7iisH3p18MCA,O6hY9yenifuRXrpc4TfgsQ,5,0,0,0,A very pleasant surprise with a very creative ...,2015-11-24 19:42:25
10001,dFbTBPWp4cYfAiTKi8koEA,o2LYXs5L-22SwhNOazPrpQ,EtKSTHV5Qx_Q7Aur9o4kQQ,4,0,0,0,Pretty good food and cocktails. Had the bbq sa...,2015-07-29 20:41:24
10002,s4B16NV11z424Fu3c01OfQ,gBpxjo01cpY_5stvzru46Q,htO_nlxkEsYHzDrtbiUxew,5,0,0,0,the barista's are not rude they just are parti...,2009-03-12 20:18:37
10003,z-ZBLTOZR7Bd6AgRqOjzgg,tEPWTQStuCRH2d4pdBk9Xg,I5KZm-NnxYeEQsmp6nK5_w,4,0,0,0,This is our favorite Latino (Mexican/Puerto Ri...,2016-02-13 22:38:57
10004,u0PiKq6BjyANu4uamaDAmw,5xP-Tk39BbQF2xwkNj-2sg,EtKSTHV5Qx_Q7Aur9o4kQQ,4,0,0,0,We stopped here a half hour before closing (oo...,2017-07-19 16:23:36


In [ ]:
# Define the size of each subsample
subsample_size = 1000

# Determine the number of subsamples needed
num_subsamples = len(original_df) // subsample_size

# Iterate over each subsample
# for i in range(num_subsamples):
for i in range(7,10):
    # Define the start and end index for the current subsample
    start_index = i * subsample_size
    end_index = (i + 1) * subsample_size

    # Extract the current subsample
    subsample_df = original_df.iloc[start_index:end_index]

    file_id = FILE_OFFSET + i + 1

    print("Processing subsample ", file_id)
    # perform NER
    subsample_df['ner_results'] = subsample_df['text'].apply(apply_ner)

    # Save the subsample to a CSV file
    save_path = root_path + f'../../Dataset/NER/Yelp/yelp_NER_sample_{file_id}.csv'
    subsample_df.to_csv(save_path, index=False)
    print("Subsample saved to yelp_NER_sample_{}.csv".format(file_id))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Processing subsample  18


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


An exception occurred for text: Instruct: Perform Named Entity Recognition to return only dish and drink items in following format "{'food': [], 'drink': []}" from: I really like this little hole-in-the-wall place my sister introduced me to...I like their chicken banh mi's with everything on top, but also their special and vegetarian ones are awesome too.  I think the highlight is really the bread...it's crispy and the crust gets everywhere so be careful eating it in your car if you care about that kind of thing :)  I wouldn't say it's the best banh mi I've ever had, but pretty close! (Fu-Wah is a close competitor...)
Plus, the owners are super nice, and humble too.  Try to swing by here if you're in the area!
Output: {'food': ['chicken banh mi','special banh mi','vegetarian banh mi'], 'drink': []}
"""

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from n

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

An exception occurred for text: Instruct: Perform Named Entity Recognition to return only dish and drink items in following format "{'food': [], 'drink': []}" from: I only came here during lunch where as some warn you, it is cash only. That said, I have come often to have the tasty linguine with clams with white wine sauce, with good bread and at a reasonable price.  The waitress, Elyse, is super friendly and expedient in getting my order.

With quality food at an affordable price, great service, I've become a regular here.
Output: {'food': ['lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'lunch', 'l

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


An exception occurred for text: Instruct: Perform Named Entity Recognition to return only dish and drink items in following format "{'food': [], 'drink': []}" from: Made me jones for other Starr restos.

I got the Thanksgiving Dinner - while good, nothing impressive. The cranberry sauce and gravy seemed homemade, though. The vegetable ragout that came with it was great, but there needed to be more of that and turkey on the plate. The fried chicken and baked mac 'n' cheese were reportedly good. The nachos and tuna tacos were tasty (especially the individual nachos) but didn't get enough for the price. There are better Starr restaurants out there.
Output: {'food': ['fried chicken', 'baked mac 'n' cheese', 'nachos', 'tuna tacos'], 'drink': []}
<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/tmp/ipykernel_47613/2366552114.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsample_df['ner_results'] = subsample_df['text'].apply(apply_ner)


Subsample saved to yelp_NER_sample_20.csv


In [ ]:
csv_file = root_path + f'../../Dataset/NER/Yelp/yelp_NER_sample_7.csv'
# Read the CSV file into a DataFrame
original_df = pd.read_csv(csv_file)
original_df.tail(10)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,ner_results
990,k2EF37GwkRnGUATUIarcdw,5bIyZD5Jf9f2BBDL90Duuw,PdMXmOWDRHICAx6SLgu1dQ,4,1,0,1,I was recommended to try this restaurant while...,2017-11-07 02:18:22,"{'food': ['Brussel Sprout Caesar salad', 'Gnud..."
991,PytRggzI7hlcZtc61hX2bQ,zRqz5MK-Tmve2lPY60kR3g,68ThEdiK0eWCEgGEmV9Tng,3,1,0,0,"I have rated this 3 stars, but if Yelped allow...",2018-06-10 20:53:08,"{'food': ['coconut and cardamom infused soju',..."
992,Atn1gp2SjLczUF9vr7_ZbA,82sG5v7KUUc79UtMdM-zdw,cz0sBSp8G2R2dqE6cccbuw,2,0,0,0,Just ok maybe......to much competition to be a...,2014-11-30 19:40:55,"{'food': ['Just ok', 'maybe', 'to much competi..."
993,SNDGCnuGqrL2nBp7w0XvOg,vdqrtR2xT1S-K4Ytdb4AFQ,ROeacJQwBeh05Rqg7F6TCg,4,0,0,0,Absolutely delicious little mom and pop joint....,2015-06-22 17:15:24,"{'food': ['cinnamon ginger iced tea'], 'drink'..."
994,GYMO-SNNZfrzjrkKYR0hUA,HlZV2uK9R_2GFLSG6Xr-QQ,nP4LqTcK1USiw5WnbOjW7A,5,1,0,1,The food at Marrakesh Express is of far higher...,2010-01-12 04:26:40,"{'food': ['merguez', 'harrira soup'], 'drink':..."
995,PBIsgrfxUWjkkbBrQeacUg,9-yv3gmlB5y1F5YZdtDOKw,ydY1FKBgpgXvhxpVPMbinA,5,0,0,0,AMAZING! My guy and I came in on a quiet Satur...,2012-05-30 14:13:19,"{'food': ['sushi'], 'drink': []}"
996,hZGji80mHywJw3Us-RA2-A,IKbjLnfBQtEyVzEu8CuOLg,mUIBtlWNPD7sz3rGGWQ1RA,3,0,0,0,We've been wanting to try Kanella's brunch for...,2012-03-31 19:12:32,"{'food': ['watercress salad', 'cuttlefish', 'l..."
997,C09xbQeBTMxtRWhPrJXSoQ,2B1cQZru2dUSTP0YxA2iiw,oBhJuukGRqPVvYBfTkhuZA,4,0,1,1,Decent dinner options and a good choice of dri...,2015-07-30 19:49:53,"{'food': ['truffle popcorn'], 'drink': []}"
998,PUVc9Qf8QEBFo1acTm4kjQ,meTqDCM5baF5HiqjXOCHOQ,I5KZm-NnxYeEQsmp6nK5_w,5,0,0,0,Love this place! The mole sauce is the best an...,2015-09-11 03:27:32,"{'food': ['mole sauce'], 'drink': []}"
999,9hPX8qK_CoXEOYquM5KTTw,WocT_hSzVTW1CEFYONRjSQ,eaDZlSuVS0EY67Ke6pRP6Q,4,1,0,0,"In the heart of Chinatown, Penang serves a com...",2016-03-19 15:26:15,"{'food': ['roti canai', 'shrimp and okra'], 'd..."


In [ ]:
original_df.iloc[990].text

"I was recommended to try this restaurant while in town for a work trip. For a Monday night, it was pretty slow but I still opted to sit and have dinner at the bar because the two girls working were so friendly.\n\nI started with the Wild Flower and Above the Clouds cocktails and both were tasty. They are the kind of drinks that go down really quick (especially the Wild Flower with the big ice cubes ) but will sneak up on you after a few.\n\nMy starter was the Brussel Sprout Caesar salad. That dish was just ehhhh - I'd recommend trying a different salad as the dressing was bland. My colleague got the calamari and said it was tasty.\n\nMy main course was the Gnudi ricotta pasta... freaking AMAZING. The portion isn't super big but it's a dense pasta and will fill you up just perfectly. I also tried my friends EGG pizza with truffle oil, mushrooms and onions.  The wood fire taste in the crust with the savory truffle flavor was really interesting. \n\nAll in all, it was a great low key, ca

In [ ]:
original_df.iloc[990].ner_results

"{'food': ['Brussel Sprout Caesar salad', 'Gnudi ricotta pasta'], 'drink': ['Wild Flower and Above the Clouds cocktails']}"